# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
load_dotenv("azure.env")

True

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 10-Jun-2025 10:23:52


## Grounding Bing Search

In [4]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [5]:
# Shoud be: /subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.CognitiveServices/accounts/<ai_service_name>/projects/<project_name>/con#nections/<connection_name>
BING_CONN_ID = os.getenv("BING_CONN_ID")

In [6]:
bing = BingGroundingTool(connection_id=BING_CONN_ID)
bing

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"

agent = project_client.create_agent(
    model="gpt-4o",
    name="Bing-grounding-agent",
    instructions=instructions,
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Agent with Grounding Bing Search",
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_jqrarwftZh6Yq5Pg8HNEuowv


In [8]:
id1 = agent.id

In [9]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_k5K7ix2iPJbzy2WRcgNpjQnz


### Example 1

In [10]:
query = "What is the current price of MICROSOFT stock?"

In [11]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_eTKaipnuISwUaQsL7w50JbkO


In [12]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


In [13]:
# Fetch run steps to get the details of the agent run
try:
    run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                bing_grounding_details = call.get("bing_grounding", {})
                if bing_grounding_details:
                    print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")
except Exception as e:
    print(f"Error fetching run steps: {e}")

try:
    usage = run.usage
    print(f"\nRun usage: {usage}")
except AttributeError:
    print("Run object has no attribute 'usage'")
except Exception as e:
    print(f"Error accessing run usage: {e}")

Step step_MAUNUCSN7aa6liIAVdqL8fpF status: completed
Step step_ZcYPiTmenoW3OW8VG9YUEhr9 status: completed
    Tool Call ID: call_G6yGcG1DqtP56n6RFiAICysP
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=current price of Microsoft stock June 6 2025

Run usage: {'prompt_tokens': 1625, 'completion_tokens': 83, 'total_tokens': 1708, 'prompt_token_details': {'cached_tokens': 0}}


In [15]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)

if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response:\n{text_message.text.value}\n")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response:
The current price of Microsoft (MSFT) stock as of June 6, 2025, is $470.38, marking a rise of $2.70 (+0.58%) from the previous trading day【3:1†source】【3:2†source】.

URL Citation: [Microsoft (MSFT) Stock Price History & Chart Since 1986](https://wallstreetnumbers.com/stocks/msft/price)
URL Citation: [Microsoft - 39 Year Stock Price History | MSFT - Macrotrends](https://www.macrotrends.net/stocks/charts/MSFT/microsoft/stock-price-history)


### Example 2

In [16]:
query = "What are the news about the Apple WWDC event?"

In [17]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_ahDcUs5puUCdnGxR8dyj5hSp


In [18]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


In [19]:
# Fetch run steps to get the details of the agent run
try:
    run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                bing_grounding_details = call.get("bing_grounding", {})
                if bing_grounding_details:
                    print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")
except Exception as e:
    print(f"Error fetching run steps: {e}")

try:
    usage = run.usage
    print(f"\nRun usage: {usage}")
except AttributeError:
    print("Run object has no attribute 'usage'")
except Exception as e:
    print(f"Error accessing run usage: {e}")

Step step_GHvPF6dLfZUsCbprhEs6f254 status: completed
Step step_12xlhoJy9vLVhy7KSVXloYjE status: completed
    Tool Call ID: call_z6RxjH7TraZIsgf2Sxfp3GI5
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=Apple WWDC 2025 news updates June 5 2025

Run usage: {'prompt_tokens': 1550, 'completion_tokens': 92, 'total_tokens': 1642, 'prompt_token_details': {'cached_tokens': 0}}


In [22]:
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)

if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response:\n{text_message.text.value}\n")

Agent response:
Apple's WWDC 2025 event has revealed several key updates, including the introduction of iOS 26, advancements in macOS, and the unveiling of a new feature named Liquid Glass. The event continues throughout the week, showcasing Apple's latest innovations【3:0†source】【3:1†source】.



### Example 3

In [23]:
query = "What are the weather informations for Mexico city? Show the report and use emojis and be funny"

In [24]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_tYxXi2wyEBowBjCaNXnd85sh


In [25]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


In [26]:
# Fetch run steps to get the details of the agent run
try:
    run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                bing_grounding_details = call.get("bing_grounding", {})
                if bing_grounding_details:
                    print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")
except Exception as e:
    print(f"Error fetching run steps: {e}")

try:
    usage = run.usage
    print(f"\nRun usage: {usage}")
except AttributeError:
    print("Run object has no attribute 'usage'")
except Exception as e:
    print(f"Error accessing run usage: {e}")

Step step_mIKbfFpaYszjSIlqevab0ZUy status: completed
Step step_RivOswRfIYSNcUZITZdOHewl status: completed
    Tool Call ID: call_Rjzcjv5pflGB9EBrdkS9htAM
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=current weather in Mexico City June 5 2025

Run usage: {'prompt_tokens': 1794, 'completion_tokens': 126, 'total_tokens': 1920, 'prompt_token_details': {'cached_tokens': 0}}


In [27]:
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)

if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response:\n{text_message.text.value}\n")

Agent response:
🌤️ Hola from Mexico City! Here's your weather report:

- Today's temperature: Breezy at 23°C (73°F) 🥳.
- 🌬️ Snuggly nights at 11°C (52°F). Bring a jacket, or maybe warmth from some tacos 🌮 😋?
- Rain? Chance at 20% – so pack an umbrella ☂️, or live your "singin’-in-the-rain" fantasy.

Stay cool, stay spicy! 🔥



### Example 4

In [40]:
query = "List the top-10 movies of Spielberg by release date"

In [41]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_4DbRCMoncww3M9hQ6WIoYqDm


In [42]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


In [43]:
# Fetch run steps to get the details of the agent run
try:
    run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                bing_grounding_details = call.get("bing_grounding", {})
                if bing_grounding_details:
                    print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")
except Exception as e:
    print(f"Error fetching run steps: {e}")

try:
    usage = run.usage
    print(f"\nRun usage: {usage}")
except AttributeError:
    print("Run object has no attribute 'usage'")
except Exception as e:
    print(f"Error accessing run usage: {e}")

Step step_HnWOgivD6La8YecWthDRXVV2 status: completed

Run usage: {'prompt_tokens': 1158, 'completion_tokens': 251, 'total_tokens': 1409, 'prompt_token_details': {'cached_tokens': 0}}


In [44]:
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)

if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response:\n{text_message.text.value}\n")

Agent response:
Here are Spielberg's **top 10 movies** (by release dates):

1. **Duel** (1971) 🚗🔥
2. **The Sugarland Express** (1974) 🚓
3. **Jaws** (1975) 🦈 - *The one that made you afraid of beaches!*
4. **Close Encounters of the Third Kind** (1977) 👽
5. **1941** (1979) 🚨 *- chaotic comedy time!*
6. **Raiders of the Lost Ark** (1981) 🏺 *Adventurer's dream begins!*
7. **E.T. the Extra-Terrestrial** (1982) 🌌👾 - *Phone home, E.T.!*
8. **Indiana Jones and the Temple of Doom** (1984) 🕶️ - *More Indie action!*
9. **The Color Purple** (1985) 💜 – An emotional rollercoaster.
10. **Empire of the Sun** (1987) ✈️🔥 – WWII with young Bale! 

Spielberg's oldies are pure gold—catch up on them if you haven't! ✨



### Post processing

In [38]:
# List all agents in the project
print("Listing all agents in the project:\n")

agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:

Agent ID: asst_jqrarwftZh6Yq5Pg8HNEuowv, Name: Bing-grounding-agent, Model: gpt-4o, Instructions: You are an AI Agent that can do some web search using Bing
Agent ID: asst_GCSPuojnFZT7jgOu93Dtx9ES, Name: Bing-grounding-agent, Model: gpt-4o, Instructions: You are an AI Agent that can do some web search using Bing
Agent ID: asst_AtWIKCQ5eNmeipBttg6b5oQa, Name: gpt-4o-mini-weather-agent, Model: gpt-4o, Instructions: You are a weather bot. Use the provided functions to help answer questions.


In [39]:
#project_client.delete_agent(id1)
#print(f"Deleted agent, agent ID: {id1}")